# Gaussian Elimination Without Partial Pivotting Manually and Scipy

In [1826]:
import math 
import numpy as np
import scipy as sp
import mpmath 
from scipy import linalg
import sympy
a=[[75. ,3. ,8. ,7. ,7. ,5. ,1. ,0. ,3. ,5. ,8. ,6. ,3. ,0. ,0],
[3. ,90. ,4. ,5. ,9. ,2. ,1. ,7. ,8. ,9. ,4. ,7. ,3. ,8. ,5],
[1. ,3. ,85. ,5. ,3. ,2. ,0. ,1. ,2. ,8. ,6. ,4. ,9. ,3. ,5],
[3. ,4. ,6. ,86. ,9. ,9. ,0. ,2. ,4. ,0. ,9. ,4. ,4. ,7. ,7],
[6. ,6. ,1. ,2. ,79. ,3. ,4. ,1. ,3. ,5. ,7. ,7. ,6. ,6. ,0],
[2. ,4. ,3. ,9. ,5. ,95. ,7. ,0. ,2. ,9. ,8. ,4. ,6. ,6. ,1],
[2. ,2. ,4. ,7. ,1. ,4. ,78. ,5. ,3. ,8. ,1. ,5. ,8. ,2. ,2],
[6. ,3. ,8. ,9. ,2. ,9. ,5. ,92. ,3. ,7. ,3. ,9. ,9. ,8. ,8],
[7. ,6. ,8. ,1. ,1. ,7. ,0. ,8. ,99. ,9. ,9. ,1. ,5. ,4. ,7],
[0. ,4. ,2. ,6. ,9. ,5. ,8. ,1. ,6. ,92. ,8. ,2. ,5. ,9. ,1],
[3. ,6. ,4. ,1. ,2. ,5. ,4. ,3. ,8. ,7. ,96. ,1. ,2. ,7. ,7],
[5. ,7. ,0. ,2. ,7. ,1. ,5. ,9. ,8. ,9. ,7. ,87. ,5. ,0. ,9],
[0. ,1. ,2. ,7. ,3. ,5. ,6. ,7. ,9. ,1. ,4. ,7. ,81. ,6. ,1],
[3. ,4. ,6. ,9. ,4. ,2. ,3. ,8. ,8. ,7. ,7. ,3. ,8. ,99. ,5],
[2. ,5. ,3. ,8. ,8. ,2. ,7. ,3. ,1. ,1. ,1. ,4. ,3. ,2. ,96]]


b=[[200.],
[300.],
[400.],
[500.],
[600.],
[700.],
[800.],
[900.],
[1000.],
[1100.],
[1200.],
[1300.],
[1400.],
[1500.],
[1600.],
]

In [1827]:
a=np.asarray(a)
b=np.asarray(b)
A = np.hstack((a,b))
np.linalg.cond(a)

2.3438827134917504

In [1828]:
%%timeit -n 15
#Gaussian Elimination (in [47])
N = A.shape[0]
for k in range(N-1):
    for i in range(k+1, N):
        r = -A[i,k] / A[k,k]
        for j in range(k+1, N+1):
            A[i,j] = A[i,j] + r * A[k,j]
        #lines below are not used during back substitution
        for j in range(k+1):
            A[i,j] = 0

680 µs ± 39.1 µs per loop (mean ± std. dev. of 7 runs, 15 loops each)


In [1829]:
%%timeit -n 15
#Back sustitution (in [54])
A[N-1,N] = A[N-1,-1] / A[N-1, -2]
for i in range(N-2, -1, -1): #2, 1, 0
    sum = 0
    for j in range(i+1, N): #i+1 to N-1
        sum = sum + A[i,j] * A[j,N]
    A[i,N] = (A[i,N] - sum)/A[i,i]
A[:,N]

63.7 µs ± 3.02 µs per loop (mean ± std. dev. of 7 runs, 15 loops each)


In [1816]:
#Checking the answers
ans = A[:,N]
ans = ans[:,np.newaxis]
np.dot(a,ans)

array([[ 200.],
       [ 300.],
       [ 400.],
       [ 500.],
       [ 600.],
       [ 700.],
       [ 800.],
       [ 900.],
       [1000.],
       [1100.],
       [1200.],
       [1300.],
       [1400.],
       [1500.],
       [1600.]])

In [1817]:
ans_sp = linalg.solve(a,b)
ans_sp

array([[-0.90631413],
       [-2.07410434],
       [-0.06027671],
       [ 0.87802894],
       [ 3.15174414],
       [ 3.11269004],
       [ 6.07000405],
       [ 3.61193444],
       [ 6.0219445 ],
       [ 7.65021476],
       [ 8.77441867],
       [ 9.81673757],
       [13.14117951],
       [10.77780735],
       [14.56126982]])

In [1818]:
a.dot(ans_sp)

array([[ 200.],
       [ 300.],
       [ 400.],
       [ 500.],
       [ 600.],
       [ 700.],
       [ 800.],
       [ 900.],
       [1000.],
       [1100.],
       [1200.],
       [1300.],
       [1400.],
       [1500.],
       [1600.]])

# Gaussian Elimination With Partial Pivotting Manually and Scipy


In [1819]:
aa=[[0. ,5. ,7. ,75. ,0. ,3. ,8. ,0. ,3. ,8. ,6. ,7. ,1. ,5. ,3],
[8. ,2. ,5. ,3. ,7. ,90. ,4. ,5. ,8. ,4. ,7. ,9. ,1. ,9. ,3],
[3. ,2. ,5. ,1. ,1. ,3. ,85. ,5. ,2. ,6. ,4. ,3. ,0. ,8. ,9],
[7. ,9. ,86. ,3. ,2. ,4. ,6. ,7. ,4. ,9. ,4. ,9. ,0. ,0. ,4],
[6. ,3. ,2. ,6. ,1. ,6. ,1. ,0. ,3. ,7. ,7. ,79. ,4. ,5. ,6],
[6. ,95. ,9. ,2. ,0. ,4. ,3. ,1. ,2. ,8. ,4. ,5. ,7. ,9. ,6],
[2. ,4. ,7. ,2. ,5. ,2. ,4. ,2. ,3. ,1. ,5. ,1. ,78. ,8. ,8],
[8. ,9. ,9. ,6. ,92. ,3. ,8. ,8. ,3. ,3. ,9. ,2. ,5. ,7. ,9],
[4. ,7. ,1. ,7. ,8. ,6. ,8. ,7. ,99. ,9. ,1. ,1. ,0. ,9. ,5],
[9. ,5. ,6. ,0. ,1. ,4. ,2. ,1. ,6. ,8. ,2. ,9. ,8. ,92. ,5],
[7. ,5. ,1. ,3. ,3. ,6. ,4. ,7. ,8. ,96. ,1. ,2. ,4. ,7. ,2],
[0. ,1. ,2. ,5. ,9. ,7. ,0. ,9. ,8. ,7. ,87. ,7. ,5. ,9. ,5],
[6. ,5. ,7. ,0. ,7. ,1. ,2. ,1. ,9. ,4. ,7. ,3. ,6. ,1. ,81],
[99. ,2. ,9. ,3. ,8. ,4. ,6. ,5. ,8. ,7. ,3. ,4. ,3. ,7. ,8],
[2. ,2. ,8. ,2. ,3. ,5. ,3. ,96. ,1. ,1. ,4. ,8. ,7. ,1. ,3]]
bb=[[200.],
[300.],
[400.],
[500.],
[600.],
[700.],
[800.],
[900.],
[1000.],
[1100.],
[1200.],
[1300.],
[1400.],
[1500.],
[1600.],
]
np.linalg.cond(aa)

2.3438827134917517

In [1820]:
aa=np.asarray(aa)
bb=np.asarray(bb)
A = np.hstack((aa,bb))
np.linalg.cond(aa)

2.3438827134917517

In [1821]:
#To ckeck if new Coeffieint Matrix can be solved or it needs pivtoing
ans_sp = linalg.solve(aa,bb)
ans_sp

array([[10.77780735],
       [ 3.11269004],
       [ 0.87802894],
       [-0.90631413],
       [ 3.61193444],
       [-2.07410434],
       [-0.06027671],
       [14.56126982],
       [ 6.0219445 ],
       [ 8.77441867],
       [ 9.81673757],
       [ 3.15174414],
       [ 6.07000405],
       [ 7.65021476],
       [13.14117951]])

In [1822]:
def partial_pivoting(a):
    A = a.copy()
    N = A.shape[0]
    p = np.eye(N,N)
    for k in range(N-1):
        maxidx = np.abs(A[k:,k]).argmax() + k #get index of the max arg
        # +k is needed, because, argmax restart at 0 for the new slice
        if (k != maxidx):
            p[[k, maxidx]] = p[[maxidx, k]]
            A[[k, maxidx]] = A[[maxidx, k]]
    return A, p

In [1823]:
A,p=partial_pivoting(A)

In [1824]:

#Gaussian Elimination (in [47])
N = A.shape[0]
for k in range(N-1):
    for i in range(k+1, N):
        r = -A[i,k] / A[k,k]
        for j in range(k+1, N+1):
            A[i,j] = A[i,j] + r * A[k,j]
        #lines below are not used during back substitution
        for j in range(k+1):
            A[i,j] = 0


In [1825]:
#Back sustitution (in [54])
A[N-1,N] = A[N-1,-1] / A[N-1, -2]
for i in range(N-2, -1, -1): #2, 1, 0
    sum = 0
    for j in range(i+1, N): #i+1 to N-1
        sum = sum + A[i,j] * A[j,N]
    A[i,N] = (A[i,N] - sum)/A[i,i]
A[:,N]  

array([10.77780735,  3.11269004,  0.87802894, -0.90631413,  3.61193444,
       -2.07410434, -0.06027671, 14.56126982,  6.0219445 ,  8.77441867,
        9.81673757,  3.15174414,  6.07000405,  7.65021476, 13.14117951])

In [1796]:
#Checking the answers
ans = A[:,N]
ans = ans[:,np.newaxis]
np.dot(aa,ans)

array([[ 1.71193951e-192],
       [-5.66455757e-194],
       [-7.06248892e-194],
       [-6.34529486e-193],
       [ 3.72920628e-193],
       [ 1.41769433e-193],
       [-1.44055132e-193],
       [ 1.94870819e-193],
       [ 1.58756245e-193],
       [-3.55890914e-194],
       [ 8.14899013e-194],
       [ 2.94791677e-194],
       [-4.30023329e-194],
       [-1.06403270e-194],
       [-4.33292554e-194]])

In [1766]:
ans_sp = linalg.solve(aa,bb)
ans_sp

array([[10.77780735],
       [ 3.11269004],
       [ 0.87802894],
       [-0.90631413],
       [ 3.61193444],
       [-2.07410434],
       [-0.06027671],
       [14.56126982],
       [ 6.0219445 ],
       [ 8.77441867],
       [ 9.81673757],
       [ 3.15174414],
       [ 6.07000405],
       [ 7.65021476],
       [13.14117951]])

# LU Decomposition Manual and Scipy without Partial Pivoting

In [1745]:
a=[[75 ,3 ,8 ,7 ,7 ,5 ,1 ,9 ,3 ,5 ,8 ,6 ,3 ,9 ,0],
[3 ,90 ,4 ,5 ,9 ,2 ,1 ,7 ,8 ,9 ,4 ,7 ,3 ,8 ,5],
[8 ,4 ,85 ,5 ,3 ,2 ,0 ,1 ,2 ,8 ,6 ,4 ,9 ,3 ,5],
[7 ,5 ,5 ,86 ,9 ,9 ,0 ,2 ,4 ,0 ,9 ,4 ,4 ,7 ,7],
[7 ,9 ,3 ,9 ,79 ,3 ,4 ,1 ,3 ,5 ,7 ,7 ,6 ,6 ,0],
[5 ,2 ,2 ,9 ,3 ,95 ,7 ,0 ,2 ,9 ,8 ,4 ,6 ,6 ,1],
[1 ,1 ,0 ,0 ,4 ,7 ,78 ,5 ,3 ,8 ,1 ,5 ,8 ,2 ,2],
[9 ,7 ,1 ,2 ,1 ,0 ,5 ,92 ,3 ,7 ,3 ,9 ,9 ,8 ,8],
[3 ,8 ,2 ,4 ,3 ,2 ,3 ,3 ,99 ,9 ,9 ,1 ,5 ,4 ,7],
[5 ,9 ,8 ,0 ,5 ,9 ,8 ,7 ,9 ,92 ,8 ,2 ,5 ,9 ,1],
[8 ,4 ,6 ,9 ,7 ,8 ,1 ,3 ,9 ,8 ,96 ,1 ,2 ,7 ,7],
[6 ,7 ,4 ,4 ,7 ,4 ,5 ,9 ,1 ,2 ,1 ,87 ,5 ,0 ,9],
[3 ,3 ,9 ,4 ,6 ,6 ,8 ,9 ,5 ,5 ,2 ,5 ,81 ,6 ,1],
[9 ,8 ,3 ,7 ,6 ,6 ,2 ,8 ,4 ,9 ,7 ,0 ,6 ,99 ,5],
[0 ,5 ,5 ,7 ,0 ,1 ,2 ,8 ,7 ,1 ,7 ,9 ,1 ,5 ,96]]
b=[[200.],
[300.],
[400.],
[500.],
[600.],
[700.],
[800.],
[900.],
[1000.],
[1100.],
[1200.],
[1300.],
[1400.],
[1500.],
[1600.],
]
B=np.asarray(a)
b=np.asarray(b)
#np.hstack((a,b))
N = B.shape[0]
A = B.copy()
for i in range(1, N):
    #loop for L
    for j in range(i): #A[a:b] is [a,b) not [a,b]
        Sum = A[0:j,j].dot(A[i,0:j])
        A[i,j] = (A[i,j] - Sum)/A[j,j]
    #loop for U
    for j in range(i,N):
        Sum = A[0:i,j].dot(A[i,0:i])
        A[i,j] = A[i,j] - Sum
U_ans = np.triu(A)
L_ans = np.tril(A)
L_ans[np.diag_indices(N)] = 1
np.linalg.cond(a)

2.586788006020457

In [1673]:
len(B[0])

15

In [1674]:
c=L_ans.dot(U_ans)-B
Check=c[:]<0.000000000001
np.all(Check==True)
#Correct

True

In [1675]:
p, l, u = sp.linalg.lu(np.asarray(a) ,permute_l = False)
print(p, end = '\n\n')
print(l, end = '\n\n')
print(u, end = '\n\n')

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]

[[ 1.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.04        1.          0.          0.          0.          0.
   0.          0.          0.          0.

In [1676]:
Y = linalg.solve(l,b)
X= linalg.solve(u,Y)
X

array([[-2.97652404],
       [-2.20258491],
       [ 0.21857529],
       [ 1.04337603],
       [ 3.13308531],
       [ 3.19149597],
       [ 5.76874833],
       [ 4.23922511],
       [ 6.21901116],
       [ 7.39811823],
       [ 8.63409752],
       [11.64041954],
       [13.09010808],
       [11.68341394],
       [13.19096656]])

# LU Decomposition Manual and Scipy with Partial Pivoting

In [1677]:
aa=[[9 ,8 ,3 ,7 ,6 ,6 ,2 ,8 ,4 ,9 ,7 ,0 ,6 ,99 ,5],
[3 ,90 ,4 ,5 ,9 ,2 ,1 ,7 ,8 ,9 ,4 ,7 ,3 ,8 ,5],
[8 ,4 ,85 ,5 ,3 ,2 ,0 ,1 ,2 ,8 ,6 ,4 ,9 ,3 ,5],
[7 ,5 ,5 ,86 ,9 ,9 ,0 ,2 ,4 ,0 ,9 ,4 ,4 ,7 ,7],
[7 ,9 ,3 ,9 ,79 ,3 ,4 ,1 ,3 ,5 ,7 ,7 ,6 ,6 ,0],
[5 ,2 ,2 ,9 ,3 ,95 ,7 ,0 ,2 ,9 ,8 ,4 ,6 ,6 ,1],
[1 ,1 ,0 ,0 ,4 ,7 ,78 ,5 ,3 ,8 ,1 ,5 ,8 ,2 ,2],
[9 ,7 ,1 ,2 ,1 ,0 ,5 ,92 ,3 ,7 ,3 ,9 ,9 ,8 ,8],
[3 ,8 ,2 ,4 ,3 ,2 ,3 ,3 ,99 ,9 ,9 ,1 ,5 ,4 ,7],
[5 ,9 ,8 ,0 ,5 ,9 ,8 ,7 ,9 ,92 ,8 ,2 ,5 ,9 ,1],
[8 ,4 ,6 ,9 ,7 ,8 ,1 ,3 ,9 ,8 ,96 ,1 ,2 ,7 ,7],
[6 ,7 ,4 ,4 ,7 ,4 ,5 ,9 ,1 ,2 ,1 ,87 ,5 ,0 ,9],
[3 ,3 ,9 ,4 ,6 ,6 ,8 ,9 ,5 ,5 ,2 ,5 ,81 ,6 ,1],
[75 ,3 ,8 ,7 ,7 ,5 ,1 ,9 ,3 ,5 ,8 ,6 ,3 ,9 ,0],
[0 ,5 ,5 ,7 ,0 ,1 ,2 ,8 ,7 ,1 ,7 ,9 ,1 ,5 ,96]]

bb=[[1500],
[300],
[400],
[500],
[600],
[700],
[800],
[900],
[1000],
[1100],
[1200],
[1300],
[1400],
[200],
[1600]]
Bc,p=partial_pivoting(np.hstack((aa,bb)))
bb=np.asarray(bb)
cc=Bc[:,N]
for i in range (N):
    bb[i,0]=cc[i]
B, p=partial_pivoting(np.asarray(aa))
np.linalg.cond(aa)

2.5867880060204564

In [1706]:
#To ckeck if new Coeffieint Matrix can be solved or it needs pivtoing
bCheck=[[1500],
[300],
[400],
[500],
[600],
[700],
[800],
[900],
[1000],
[1100],
[1200],
[1300],
[1400],
[200],
[1600]]
p, l, u = sp.linalg.lu(np.asarray(aa) ,permute_l = False)
Y = linalg.solve(l,np.asarray(bCheck))
X= linalg.solve(u,Y)
X
#Results are not correct

array([[16.52493991],
       [-1.34074943],
       [-1.16884424],
       [ 0.78273543],
       [ 2.61130009],
       [ 3.14720741],
       [ 5.91209082],
       [ 3.53886669],
       [ 6.13686196],
       [ 7.94290996],
       [ 8.12161241],
       [10.25774653],
       [13.77099822],
       [-3.2456863 ],
       [14.23095814]])

In [1680]:
%%timeit -n 15
N = B.shape[0]
A = B.copy()
for i in range(1, N):
    #loop for L
    for j in range(i): #A[a:b] is [a,b) not [a,b]
        Sum = A[0:j,j].dot(A[i,0:j])
        A[i,j] = (A[i,j] - Sum)/A[j,j]
    #loop for U
    for j in range(i,N):
        Sum = A[0:i,j].dot(A[i,0:i])
        A[i,j] = A[i,j] - Sum
U_ans = np.triu(A)
L_ans = np.tril(A)
L_ans[np.diag_indices(N)] = 1


333 µs ± 21.2 µs per loop (mean ± std. dev. of 7 runs, 15 loops each)


In [1681]:
c=L_ans.dot(U_ans)-B
Check=c[:]<0.000000000001
np.all(Check==True)

True

In [1652]:
%%timeit -n 15
p, l, u = sp.linalg.lu(np.asarray(B) ,permute_l = False)

12 µs ± 4.48 µs per loop (mean ± std. dev. of 7 runs, 15 loops each)


In [1651]:
#For LU method, when Scipy function is used, it is 3.5 times faster

In [1637]:
B,p=partial_pivoting(np.asarray(aa))
p, l, u = sp.linalg.lu(np.asarray(B) ,permute_l = False)
print(p, end = '\n\n')
print(l, end = '\n\n')
print(u, end = '\n\n')

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]

[[ 1.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.04        1.          0.          0.          0.          0.
   0.          0.          0.          0.

In [1586]:
Y = linalg.solve(l,bb)
X= linalg.solve(u,Y)
X

array([[-2.97652404],
       [-2.20258491],
       [ 0.21857529],
       [ 1.04337603],
       [ 3.13308531],
       [ 3.19149597],
       [ 5.76874833],
       [ 4.23922511],
       [ 6.21901116],
       [ 7.39811823],
       [ 8.63409752],
       [11.64041954],
       [13.09010808],
       [11.68341394],
       [13.19096656]])

# Cholesky Method Manual and Scipy Without Partial Pivoting


In [1587]:
def cholesky_user(A):
    N = A.shape[0]
    L = np.zeros([N,N])
    for i in range(N):
        for j in range(i+1):
            if i == j:
                Sum = 0
                for k in range(j):
                    Sum = Sum + L[j,k]**2
                try:
                    L[i,j] = (A[i,i] - Sum)**0.5
                except Exception as e:
                    print(e)
                    return None
            else:
                Sum = 0
                for k in range(j):
                    Sum = Sum + L[i,k]*L[j,k]
                L[i,j] = (A[i,j] - Sum) / L[j,j]
    return L

In [1588]:
a=[[75 ,3 ,8 ,7 ,7 ,5 ,1 ,9 ,3 ,5 ,8 ,6 ,3 ,9 ,0],
[3 ,90 ,4 ,5 ,9 ,2 ,1 ,7 ,8 ,9 ,4 ,7 ,3 ,8 ,5],
[8 ,4 ,85 ,5 ,3 ,2 ,0 ,1 ,2 ,8 ,6 ,4 ,9 ,3 ,5],
[7 ,5 ,5 ,86 ,9 ,9 ,0 ,2 ,4 ,0 ,9 ,4 ,4 ,7 ,7],
[7 ,9 ,3 ,9 ,79 ,3 ,4 ,1 ,3 ,5 ,7 ,7 ,6 ,6 ,0],
[5 ,2 ,2 ,9 ,3 ,95 ,7 ,0 ,2 ,9 ,8 ,4 ,6 ,6 ,1],
[1 ,1 ,0 ,0 ,4 ,7 ,78 ,5 ,3 ,8 ,1 ,5 ,8 ,2 ,2],
[9 ,7 ,1 ,2 ,1 ,0 ,5 ,92 ,3 ,7 ,3 ,9 ,9 ,8 ,8],
[3 ,8 ,2 ,4 ,3 ,2 ,3 ,3 ,99 ,9 ,9 ,1 ,5 ,4 ,7],
[5 ,9 ,8 ,0 ,5 ,9 ,8 ,7 ,9 ,92 ,8 ,2 ,5 ,9 ,1],
[8 ,4 ,6 ,9 ,7 ,8 ,1 ,3 ,9 ,8 ,96 ,1 ,2 ,7 ,7],
[6 ,7 ,4 ,4 ,7 ,4 ,5 ,9 ,1 ,2 ,1 ,87 ,5 ,0 ,9],
[3 ,3 ,9 ,4 ,6 ,6 ,8 ,9 ,5 ,5 ,2 ,5 ,81 ,6 ,1],
[9 ,8 ,3 ,7 ,6 ,6 ,2 ,8 ,4 ,9 ,7 ,0 ,6 ,99 ,5],
[0 ,5 ,5 ,7 ,0 ,1 ,2 ,8 ,7 ,1 ,7 ,9 ,1 ,5 ,96]]
b=[[200.],
[300.],
[400.],
[500.],
[600.],
[700.],
[800.],
[900.],
[1000.],
[1100.],
[1200.],
[1300.],
[1400.],
[1500.],
[1600.],
]

a=np.asarray(a)
b=np.asarray(b)
np.linalg.cond(a)

2.586788006020457

In [1589]:
#User Defined Method is correct
Lower=cholesky_user(a)
c=Lower.dot(Lower.T)-a
Check=c[:]<0.000000000001
np.all(Check==True)

True

In [1590]:
Lower=cholesky_user(a)
Higher=[[0. for i in range(0,a.shape[0])] for i in range(0,a.shape[0])]
Higher=np.asarray(Higher)
for i in range(0,15):
    for j in range(0,15):
        Higher[j,i]=Lower[i,j]
Y = linalg.solve(Lower,b)
X= linalg.solve(Higher,Y)
X

array([[-2.97652404],
       [-2.20258491],
       [ 0.21857529],
       [ 1.04337603],
       [ 3.13308531],
       [ 3.19149597],
       [ 5.76874833],
       [ 4.23922511],
       [ 6.21901116],
       [ 7.39811823],
       [ 8.63409752],
       [11.64041954],
       [13.09010808],
       [11.68341394],
       [13.19096656]])

In [1591]:
a.dot(X)

array([[ 200.],
       [ 300.],
       [ 400.],
       [ 500.],
       [ 600.],
       [ 700.],
       [ 800.],
       [ 900.],
       [1000.],
       [1100.],
       [1200.],
       [1300.],
       [1400.],
       [1500.],
       [1600.]])

In [1592]:
LowerCh=sp.linalg.cholesky(a, lower=True)
UpperCh=sp.linalg.cholesky(np.asarray(a))


In [1593]:
c=LowerCh.dot(LowerCh.T)-a
Check=c[:]<0.000000000001
np.all(Check==True)
#Correct!

True

In [1594]:
Y = linalg.solve(LowerCh,b)
X= linalg.solve(UpperCh,Y)
X
#Checked with EXCEL and it is correct

array([[-2.97652404],
       [-2.20258491],
       [ 0.21857529],
       [ 1.04337603],
       [ 3.13308531],
       [ 3.19149597],
       [ 5.76874833],
       [ 4.23922511],
       [ 6.21901116],
       [ 7.39811823],
       [ 8.63409752],
       [11.64041954],
       [13.09010808],
       [11.68341394],
       [13.19096656]])

# Cholesky Method Manual and Scipy With Partial Pivoting

In [1684]:
aa=[[9 ,8 ,3 ,7 ,6 ,6 ,2 ,8 ,4 ,9 ,7 ,0 ,6 ,99 ,5],
[3 ,90 ,4 ,5 ,9 ,2 ,1 ,7 ,8 ,9 ,4 ,7 ,3 ,8 ,5],
[8 ,4 ,85 ,5 ,3 ,2 ,0 ,1 ,2 ,8 ,6 ,4 ,9 ,3 ,5],
[7 ,5 ,5 ,86 ,9 ,9 ,0 ,2 ,4 ,0 ,9 ,4 ,4 ,7 ,7],
[7 ,9 ,3 ,9 ,79 ,3 ,4 ,1 ,3 ,5 ,7 ,7 ,6 ,6 ,0],
[5 ,2 ,2 ,9 ,3 ,95 ,7 ,0 ,2 ,9 ,8 ,4 ,6 ,6 ,1],
[1 ,1 ,0 ,0 ,4 ,7 ,78 ,5 ,3 ,8 ,1 ,5 ,8 ,2 ,2],
[9 ,7 ,1 ,2 ,1 ,0 ,5 ,92 ,3 ,7 ,3 ,9 ,9 ,8 ,8],
[3 ,8 ,2 ,4 ,3 ,2 ,3 ,3 ,99 ,9 ,9 ,1 ,5 ,4 ,7],
[5 ,9 ,8 ,0 ,5 ,9 ,8 ,7 ,9 ,92 ,8 ,2 ,5 ,9 ,1],
[8 ,4 ,6 ,9 ,7 ,8 ,1 ,3 ,9 ,8 ,96 ,1 ,2 ,7 ,7],
[6 ,7 ,4 ,4 ,7 ,4 ,5 ,9 ,1 ,2 ,1 ,87 ,5 ,0 ,9],
[3 ,3 ,9 ,4 ,6 ,6 ,8 ,9 ,5 ,5 ,2 ,5 ,81 ,6 ,1],
[75 ,3 ,8 ,7 ,7 ,5 ,1 ,9 ,3 ,5 ,8 ,6 ,3 ,9 ,0],
[0 ,5 ,5 ,7 ,0 ,1 ,2 ,8 ,7 ,1 ,7 ,9 ,1 ,5 ,96]]

bb=[[1500],
[300],
[400],
[500],
[600],
[700],
[800],
[900],
[1000],
[1100],
[1200],
[1300],
[1400],
[200],
[1600]]
Bc,p=partial_pivoting(np.hstack((aa,bb)))
bb=np.asarray(bb)
cc=Bc[:,N]
for i in range (N):
    bb[i,0]=cc[i]
np.linalg.cond(aa)
B, p=partial_pivoting(np.asarray(aa))

In [1707]:
#To ckeck if new Coeffieint Matrix can be solved or it needs pivtoing
bCheck=[[1500],
[300],
[400],
[500],
[600],
[700],
[800],
[900],
[1000],
[1100],
[1200],
[1300],
[1400],
[200],
[1600]]
LowerCh=sp.linalg.cholesky(np.asarray(aa), lower=True)
UpperCh=sp.linalg.cholesky(np.asarray(aa))
Y = linalg.solve(LowerCh,bCheck)
X= linalg.solve(UpperCh,Y)
X
# It cannot be performed without partial pivoting

LinAlgError: 14-th leading minor of the array is not positive definite

In [1596]:
Lower=cholesky_user(B)
c=Lower.dot(Lower.T)-B
Check=c[:]<0.000000000001
np.all(Check==True)

True

In [1634]:
%%timeit -n 15
Lower=cholesky_user(B)

586 µs ± 32.2 µs per loop (mean ± std. dev. of 7 runs, 15 loops each)


In [1632]:
Lower=cholesky_user(B)
Higher=[[0. for i in range(0,a.shape[0])] for i in range(0,a.shape[0])]
Higher=np.asarray(Higher)
for i in range(0,15):
    for j in range(0,15):
        Higher[j,i]=Lower[i,j]
Y = linalg.solve(Lower,b)
X= linalg.solve(Higher,Y)
X

array([[-2.97652404],
       [-2.20258491],
       [ 0.21857529],
       [ 1.04337603],
       [ 3.13308531],
       [ 3.19149597],
       [ 5.76874833],
       [ 4.23922511],
       [ 6.21901116],
       [ 7.39811823],
       [ 8.63409752],
       [11.64041954],
       [13.09010808],
       [11.68341394],
       [13.19096656]])

In [1599]:

LowerCh=sp.linalg.cholesky(B, lower=True)
UpperCh=sp.linalg.cholesky(B)
c=LowerCh.dot(LowerCh.T)-B
Check=c[:]<0.000000000001
np.all(Check==True)
#Correct!

True

In [1649]:
%%timeit -n 15
LowerCh=sp.linalg.cholesky(B, lower=True)

11.8 µs ± 1.61 µs per loop (mean ± std. dev. of 7 runs, 15 loops each)


In [1620]:
#After comparing the runtime of cholesky method using defined function and Scipy function. It was found out that Scipy function
#is 32 times faster
#And when cholesky is compared with LU method. LU was faster when it was defined by user () and when it was called by Scipy

In [1602]:
Y = linalg.solve(LowerCh,b)
X= linalg.solve(UpperCh,Y)
X

array([[-2.97652404],
       [-2.20258491],
       [ 0.21857529],
       [ 1.04337603],
       [ 3.13308531],
       [ 3.19149597],
       [ 5.76874833],
       [ 4.23922511],
       [ 6.21901116],
       [ 7.39811823],
       [ 8.63409752],
       [11.64041954],
       [13.09010808],
       [11.68341394],
       [13.19096656]])

# Gauss-Jordan method Manual and Scipy Without Partial Pivoting

In [1603]:
a=[[75 ,3 ,8 ,7 ,7 ,5 ,1 ,9 ,3 ,5 ,8 ,6 ,3 ,9 ,0],
[3 ,90 ,4 ,5 ,9 ,2 ,1 ,7 ,8 ,9 ,4 ,7 ,3 ,8 ,5],
[8 ,4 ,85 ,5 ,3 ,2 ,0 ,1 ,2 ,8 ,6 ,4 ,9 ,3 ,5],
[7 ,5 ,5 ,86 ,9 ,9 ,0 ,2 ,4 ,0 ,9 ,4 ,4 ,7 ,7],
[7 ,9 ,3 ,9 ,79 ,3 ,4 ,1 ,3 ,5 ,7 ,7 ,6 ,6 ,0],
[5 ,2 ,2 ,9 ,3 ,95 ,7 ,0 ,2 ,9 ,8 ,4 ,6 ,6 ,1],
[1 ,1 ,0 ,0 ,4 ,7 ,78 ,5 ,3 ,8 ,1 ,5 ,8 ,2 ,2],
[9 ,7 ,1 ,2 ,1 ,0 ,5 ,92 ,3 ,7 ,3 ,9 ,9 ,8 ,8],
[3 ,8 ,2 ,4 ,3 ,2 ,3 ,3 ,99 ,9 ,9 ,1 ,5 ,4 ,7],
[5 ,9 ,8 ,0 ,5 ,9 ,8 ,7 ,9 ,92 ,8 ,2 ,5 ,9 ,1],
[8 ,4 ,6 ,9 ,7 ,8 ,1 ,3 ,9 ,8 ,96 ,1 ,2 ,7 ,7],
[6 ,7 ,4 ,4 ,7 ,4 ,5 ,9 ,1 ,2 ,1 ,87 ,5 ,0 ,9],
[3 ,3 ,9 ,4 ,6 ,6 ,8 ,9 ,5 ,5 ,2 ,5 ,81 ,6 ,1],
[9 ,8 ,3 ,7 ,6 ,6 ,2 ,8 ,4 ,9 ,7 ,0 ,6 ,99 ,5],
[0 ,5 ,5 ,7 ,0 ,1 ,2 ,8 ,7 ,1 ,7 ,9 ,1 ,5 ,96]]
b=[[200.],
[300.],
[400.],
[500.],
[600.],
[700.],
[800.],
[900.],
[1000.],
[1100.],
[1200.],
[1300.],
[1400.],
[1500.],
[1600.],
]

a=np.asarray(a)
b=np.asarray(b)
np.linalg.cond(a)

2.586788006020457

In [1604]:
## Making function out of the tested code
def GJ_inv(a):
    N = a.shape[0]
    I = np.eye(N,N)
    A = np.hstack([a,I])
    for k in range(N-1):
        for i in range(k+1, N):
            r = -A[i,k] / A[k,k]
            for j in range(k+1, N+k+1): #optimize: 0, 2N to k+1, N+k+1
                A[i,j] = A[i,j] + r * A[k,j]
    for k in range(N-1,-1,-1):
        for i in range(k-1, -1, -1):
            r = -A[i,k] / A[k,k]
            for j in range(k+1, 2*N): #optimize by change 0 to k+1
                A[i,j] = A[i,j] + r * A[k,j]
    for i in range(N):
        for j in range(N,2*N):
            A[i,j] = A[i,j]/A[i,i]
    return A[:,N:]

In [1605]:
Inv = GJ_inv(a)
X=Inv.dot(b)
X

array([[-2.97652404],
       [-2.20258491],
       [ 0.21857529],
       [ 1.04337603],
       [ 3.13308531],
       [ 3.19149597],
       [ 5.76874833],
       [ 4.23922511],
       [ 6.21901116],
       [ 7.39811823],
       [ 8.63409752],
       [11.64041954],
       [13.09010808],
       [11.68341394],
       [13.19096656]])

In [1606]:
Inv=sp.linalg.inv(a)
X=Inv.dot(b)
X

array([[-2.97652404],
       [-2.20258491],
       [ 0.21857529],
       [ 1.04337603],
       [ 3.13308531],
       [ 3.19149597],
       [ 5.76874833],
       [ 4.23922511],
       [ 6.21901116],
       [ 7.39811823],
       [ 8.63409752],
       [11.64041954],
       [13.09010808],
       [11.68341394],
       [13.19096656]])

# Gauss-Jordan method Manual and Scipy With Partial Pivoting

In [1607]:
aa=[[9 ,8 ,3 ,7 ,6 ,6 ,2 ,8 ,4 ,9 ,7 ,0 ,6 ,99 ,5],
[3 ,90 ,4 ,5 ,9 ,2 ,1 ,7 ,8 ,9 ,4 ,7 ,3 ,8 ,5],
[8 ,4 ,85 ,5 ,3 ,2 ,0 ,1 ,2 ,8 ,6 ,4 ,9 ,3 ,5],
[7 ,5 ,5 ,86 ,9 ,9 ,0 ,2 ,4 ,0 ,9 ,4 ,4 ,7 ,7],
[7 ,9 ,3 ,9 ,79 ,3 ,4 ,1 ,3 ,5 ,7 ,7 ,6 ,6 ,0],
[5 ,2 ,2 ,9 ,3 ,95 ,7 ,0 ,2 ,9 ,8 ,4 ,6 ,6 ,1],
[1 ,1 ,0 ,0 ,4 ,7 ,78 ,5 ,3 ,8 ,1 ,5 ,8 ,2 ,2],
[9 ,7 ,1 ,2 ,1 ,0 ,5 ,92 ,3 ,7 ,3 ,9 ,9 ,8 ,8],
[3 ,8 ,2 ,4 ,3 ,2 ,3 ,3 ,99 ,9 ,9 ,1 ,5 ,4 ,7],
[5 ,9 ,8 ,0 ,5 ,9 ,8 ,7 ,9 ,92 ,8 ,2 ,5 ,9 ,1],
[8 ,4 ,6 ,9 ,7 ,8 ,1 ,3 ,9 ,8 ,96 ,1 ,2 ,7 ,7],
[6 ,7 ,4 ,4 ,7 ,4 ,5 ,9 ,1 ,2 ,1 ,87 ,5 ,0 ,9],
[3 ,3 ,9 ,4 ,6 ,6 ,8 ,9 ,5 ,5 ,2 ,5 ,81 ,6 ,1],
[75 ,3 ,8 ,7 ,7 ,5 ,1 ,9 ,3 ,5 ,8 ,6 ,3 ,9 ,0],
[0 ,5 ,5 ,7 ,0 ,1 ,2 ,8 ,7 ,1 ,7 ,9 ,1 ,5 ,96]]

bb=[[1500],
[300],
[400],
[500],
[600],
[700],
[800],
[900],
[1000],
[1100],
[1200],
[1300],
[1400],
[200],
[1600]]
Bc,p=partial_pivoting(np.hstack((aa,bb)))
bb=np.asarray(bb)
cc=Bc[:,N]
for i in range (N):
    bb[i,0]=cc[i]
np.linalg.cond(aa)

2.5867880060204564

In [1701]:
bCheck=[[1500],
[300],
[400],
[500],
[600],
[700],
[800],
[900],
[1000],
[1100],
[1200],
[1300],
[1400],
[200],
[1600]]
Inv=sp.linalg.inv(np.asarray(aa))
X=Inv.dot(np.asarray(bCheck))
X
#Correct

array([[-2.97652404],
       [-2.20258491],
       [ 0.21857529],
       [ 1.04337603],
       [ 3.13308531],
       [ 3.19149597],
       [ 5.76874833],
       [ 4.23922511],
       [ 6.21901116],
       [ 7.39811823],
       [ 8.63409752],
       [11.64041954],
       [13.09010808],
       [11.68341394],
       [13.19096656]])

In [1608]:
B,p=partial_pivoting(np.asarray(aa))

In [1609]:
Inv = GJ_inv(B)
X=Inv.dot(cc)
X

array([-2.97652404, -2.20258491,  0.21857529,  1.04337603,  3.13308531,
        3.19149597,  5.76874833,  4.23922511,  6.21901116,  7.39811823,
        8.63409752, 11.64041954, 13.09010808, 11.68341394, 13.19096656])

In [1610]:
Inv=sp.linalg.inv(B)
X=Inv.dot(cc)
X

array([-2.97652404, -2.20258491,  0.21857529,  1.04337603,  3.13308531,
        3.19149597,  5.76874833,  4.23922511,  6.21901116,  7.39811823,
        8.63409752, 11.64041954, 13.09010808, 11.68341394, 13.19096656])

# Jacobi iterative Manual Without Partial Pivoting

In [1692]:
a=[[75 ,3 ,8 ,7 ,7 ,5 ,1 ,9 ,3 ,5 ,8 ,6 ,3 ,9 ,0],
[3 ,90 ,4 ,5 ,9 ,2 ,1 ,7 ,8 ,9 ,4 ,7 ,3 ,8 ,5],
[8 ,4 ,85 ,5 ,3 ,2 ,0 ,1 ,2 ,8 ,6 ,4 ,9 ,3 ,5],
[7 ,5 ,5 ,86 ,9 ,9 ,0 ,2 ,4 ,0 ,9 ,4 ,4 ,7 ,7],
[7 ,9 ,3 ,9 ,79 ,3 ,4 ,1 ,3 ,5 ,7 ,7 ,6 ,6 ,0],
[5 ,2 ,2 ,9 ,3 ,95 ,7 ,0 ,2 ,9 ,8 ,4 ,6 ,6 ,1],
[1 ,1 ,0 ,0 ,4 ,7 ,78 ,5 ,3 ,8 ,1 ,5 ,8 ,2 ,2],
[9 ,7 ,1 ,2 ,1 ,0 ,5 ,92 ,3 ,7 ,3 ,9 ,9 ,8 ,8],
[3 ,8 ,2 ,4 ,3 ,2 ,3 ,3 ,99 ,9 ,9 ,1 ,5 ,4 ,7],
[5 ,9 ,8 ,0 ,5 ,9 ,8 ,7 ,9 ,92 ,8 ,2 ,5 ,9 ,1],
[8 ,4 ,6 ,9 ,7 ,8 ,1 ,3 ,9 ,8 ,96 ,1 ,2 ,7 ,7],
[6 ,7 ,4 ,4 ,7 ,4 ,5 ,9 ,1 ,2 ,1 ,87 ,5 ,0 ,9],
[3 ,3 ,9 ,4 ,6 ,6 ,8 ,9 ,5 ,5 ,2 ,5 ,81 ,6 ,1],
[9 ,8 ,3 ,7 ,6 ,6 ,2 ,8 ,4 ,9 ,7 ,0 ,6 ,99 ,5],
[0 ,5 ,5 ,7 ,0 ,1 ,2 ,8 ,7 ,1 ,7 ,9 ,1 ,5 ,96]]
b=[[200.],
[300.],
[400.],
[500.],
[600.],
[700.],
[800.],
[900.],
[1000.],
[1100.],
[1200.],
[1300.],
[1400.],
[1500.],
[1600.],
]

a=np.asarray(a)
b=np.asarray(b)
np.linalg.cond(a)

2.586788006020457

In [1693]:
def jacobi(A,b):
    """This function use Jacobi method to solve Ax = b
    A is decomposed into D + R where D is the diagonal term and R is the rest
    D^-1 is 1/a_ii for all diagonal terms
    Function input: A = nxn numpy array, b = nx1 numpy array
    This function return x
    This function is not the most efficient way of Jacobi method
    It is just to show the algorithm and how it works"""
    N = A.shape[0]
    D_inv = np.zeros((N,N))
    D_inv[range(N),range(N)] = 1/A[range(N),range(N)]
    R = A.copy()
    R[range(N),range(N)] = 0
    x = np.ones((N,1))
    for i in range(2000):
        x_old = x
        x = D_inv.dot(b-R.dot(x))
        diff = np.abs(x_old-x)
        norm_av = sp.linalg.norm(diff) / N**2
        if norm_av < 1e-14:
            break
        if i%5 == 0:
            print(x,end = '\n\n')
        
    print('#iteration = ', i, 'diff_av = ', norm_av)
    return x

In [1694]:
ans = jacobi(a,b)

[[ 1.68      ]
 [ 2.5       ]
 [ 4.        ]
 [ 4.97674419]
 [ 6.70886076]
 [ 6.69473684]
 [ 9.65384615]
 [ 9.        ]
 [ 9.46464646]
 [11.0326087 ]
 [11.66666667]
 [14.20689655]
 [16.39506173]
 [14.34343434]
 [16.0625    ]]

[[-4.40770317]
 [-3.40194351]
 [-0.84469326]
 [-0.15972357]
 [ 1.82718449]
 [ 2.19500213]
 [ 4.89669393]
 [ 3.10808889]
 [ 5.28997778]
 [ 6.10265274]
 [ 7.43674389]
 [10.59555865]
 [11.84699887]
 [10.48735503]
 [12.34784732]]

[[-2.52232724]
 [-1.82189661]
 [ 0.55619816]
 [ 1.42519776]
 [ 3.54768055]
 [ 3.50789145]
 [ 6.04558295]
 [ 4.59838799]
 [ 6.51394243]
 [ 7.80917361]
 [ 9.01411359]
 [11.97197425]
 [13.48453792]
 [12.06304341]
 [13.45855062]]

[[-3.12068794]
 [-2.32341685]
 [ 0.11141267]
 [ 0.92218432]
 [ 3.00149132]
 [ 3.09107095]
 [ 5.68088001]
 [ 4.12522561]
 [ 6.12539894]
 [ 7.26764758]
 [ 8.51347896]
 [11.53518279]
 [12.96491441]
 [11.56291814]
 [13.10603431]]

[[-2.93076587]
 [-2.1642324 ]
 [ 0.25258911]
 [ 1.08184273]
 [ 3.17485374]
 [ 3.22337125]
 [

# Jacobi iterative Manual With Partial Pivoting

In [1715]:
aa=[[9 ,8 ,3 ,7 ,6 ,6 ,2 ,8 ,4 ,9 ,7 ,0 ,6 ,99 ,5],
[3 ,90 ,4 ,5 ,9 ,2 ,1 ,7 ,8 ,9 ,4 ,7 ,3 ,8 ,5],
[8 ,4 ,85 ,5 ,3 ,2 ,0 ,1 ,2 ,8 ,6 ,4 ,9 ,3 ,5],
[7 ,5 ,5 ,86 ,9 ,9 ,0 ,2 ,4 ,0 ,9 ,4 ,4 ,7 ,7],
[7 ,9 ,3 ,9 ,79 ,3 ,4 ,1 ,3 ,5 ,7 ,7 ,6 ,6 ,0],
[5 ,2 ,2 ,9 ,3 ,95 ,7 ,0 ,2 ,9 ,8 ,4 ,6 ,6 ,1],
[1 ,1 ,0 ,0 ,4 ,7 ,78 ,5 ,3 ,8 ,1 ,5 ,8 ,2 ,2],
[9 ,7 ,1 ,2 ,1 ,0 ,5 ,92 ,3 ,7 ,3 ,9 ,9 ,8 ,8],
[3 ,8 ,2 ,4 ,3 ,2 ,3 ,3 ,99 ,9 ,9 ,1 ,5 ,4 ,7],
[5 ,9 ,8 ,0 ,5 ,9 ,8 ,7 ,9 ,92 ,8 ,2 ,5 ,9 ,1],
[8 ,4 ,6 ,9 ,7 ,8 ,1 ,3 ,9 ,8 ,96 ,1 ,2 ,7 ,7],
[6 ,7 ,4 ,4 ,7 ,4 ,5 ,9 ,1 ,2 ,1 ,87 ,5 ,0 ,9],
[3 ,3 ,9 ,4 ,6 ,6 ,8 ,9 ,5 ,5 ,2 ,5 ,81 ,6 ,1],
[75 ,3 ,8 ,7 ,7 ,5 ,1 ,9 ,3 ,5 ,8 ,6 ,3 ,9 ,0],
[0 ,5 ,5 ,7 ,0 ,1 ,2 ,8 ,7 ,1 ,7 ,9 ,1 ,5 ,96]]

bb=[[1500],
[300],
[400],
[500],
[600],
[700],
[800],
[900],
[1000],
[1100],
[1200],
[1300],
[1400],
[200],
[1600]]
Bc,p=partial_pivoting(np.hstack((aa,bb)))
bb=np.asarray(bb)
cc=Bc[:,N]
for i in range (N):
    bb[i,0]=cc[i]
np.linalg.cond(aa)

2.5867880060204564

In [1716]:
#To ckeck if new Coeffieint Matrix can be solved or it needs pivtoing
bCheck=[[1500],
[300],
[400],
[500],
[600],
[700],
[800],
[900],
[1000],
[1100],
[1200],
[1300],
[1400],
[200],
[1600]]
ans = jacobi(aa,np.asarray(bCheck))
#it cannot be performed and it needs partial pivotting

AttributeError: 'list' object has no attribute 'shape'

In [1717]:
B,p=partial_pivoting(np.asarray(aa))
ans = jacobi(B,bb)

[[ 1.68      ]
 [ 2.5       ]
 [ 4.        ]
 [ 4.97674419]
 [ 6.70886076]
 [ 6.69473684]
 [ 9.65384615]
 [ 9.        ]
 [ 9.46464646]
 [11.0326087 ]
 [11.66666667]
 [14.20689655]
 [16.39506173]
 [14.34343434]
 [16.0625    ]]

[[-4.40770317]
 [-3.40194351]
 [-0.84469326]
 [-0.15972357]
 [ 1.82718449]
 [ 2.19500213]
 [ 4.89669393]
 [ 3.10808889]
 [ 5.28997778]
 [ 6.10265274]
 [ 7.43674389]
 [10.59555865]
 [11.84699887]
 [10.48735503]
 [12.34784732]]

[[-2.52232724]
 [-1.82189661]
 [ 0.55619816]
 [ 1.42519776]
 [ 3.54768055]
 [ 3.50789145]
 [ 6.04558295]
 [ 4.59838799]
 [ 6.51394243]
 [ 7.80917361]
 [ 9.01411359]
 [11.97197425]
 [13.48453792]
 [12.06304341]
 [13.45855062]]

[[-3.12068794]
 [-2.32341685]
 [ 0.11141267]
 [ 0.92218432]
 [ 3.00149132]
 [ 3.09107095]
 [ 5.68088001]
 [ 4.12522561]
 [ 6.12539894]
 [ 7.26764758]
 [ 8.51347896]
 [11.53518279]
 [12.96491441]
 [11.56291814]
 [13.10603431]]

[[-2.93076587]
 [-2.1642324 ]
 [ 0.25258911]
 [ 1.08184273]
 [ 3.17485374]
 [ 3.22337125]
 [

#  Gauss-Seidel method Manual Without Partial Pivoting

In [1616]:
def gauss_seidel(A,b, maxit = 200000):
    N = A.shape[0]
    x = np.ones((N,1))
    L = np.tril(A)
    U = np.triu(A, k = 1)
    for i in range(maxit):
        x_old = x
        #L x = rhs
        rhs = b - U.dot(x)        
        x = forward_sub(L,rhs,N)
        diff = (((x-x_old)**2).sum())**0.5
        if diff < 1e-10:
            print('total iteration = ', i)
            print('diff = ', diff)
            SSE = ((A.dot(x) - b)**2).sum()
            print('SSE = ', SSE)
            break
    return x

def forward_sub(L,b,N):
    y = np.empty((N,1)) #(N,1) is needed here
    for i in range(N):  #to return column vector
        Sum = 0
        for k in range(i):
            Sum = Sum + L[i,k] * y[k]
        y[i] = (b[i] - Sum)/L[i,i]
    return y

In [1617]:
a=[[75 ,3 ,8 ,7 ,7 ,5 ,1 ,9 ,3 ,5 ,8 ,6 ,3 ,9 ,0],
[3 ,90 ,4 ,5 ,9 ,2 ,1 ,7 ,8 ,9 ,4 ,7 ,3 ,8 ,5],
[8 ,4 ,85 ,5 ,3 ,2 ,0 ,1 ,2 ,8 ,6 ,4 ,9 ,3 ,5],
[7 ,5 ,5 ,86 ,9 ,9 ,0 ,2 ,4 ,0 ,9 ,4 ,4 ,7 ,7],
[7 ,9 ,3 ,9 ,79 ,3 ,4 ,1 ,3 ,5 ,7 ,7 ,6 ,6 ,0],
[5 ,2 ,2 ,9 ,3 ,95 ,7 ,0 ,2 ,9 ,8 ,4 ,6 ,6 ,1],
[1 ,1 ,0 ,0 ,4 ,7 ,78 ,5 ,3 ,8 ,1 ,5 ,8 ,2 ,2],
[9 ,7 ,1 ,2 ,1 ,0 ,5 ,92 ,3 ,7 ,3 ,9 ,9 ,8 ,8],
[3 ,8 ,2 ,4 ,3 ,2 ,3 ,3 ,99 ,9 ,9 ,1 ,5 ,4 ,7],
[5 ,9 ,8 ,0 ,5 ,9 ,8 ,7 ,9 ,92 ,8 ,2 ,5 ,9 ,1],
[8 ,4 ,6 ,9 ,7 ,8 ,1 ,3 ,9 ,8 ,96 ,1 ,2 ,7 ,7],
[6 ,7 ,4 ,4 ,7 ,4 ,5 ,9 ,1 ,2 ,1 ,87 ,5 ,0 ,9],
[3 ,3 ,9 ,4 ,6 ,6 ,8 ,9 ,5 ,5 ,2 ,5 ,81 ,6 ,1],
[9 ,8 ,3 ,7 ,6 ,6 ,2 ,8 ,4 ,9 ,7 ,0 ,6 ,99 ,5],
[0 ,5 ,5 ,7 ,0 ,1 ,2 ,8 ,7 ,1 ,7 ,9 ,1 ,5 ,96]]
b=[[200.],
[300.],
[400.],
[500.],
[600.],
[700.],
[800.],
[900.],
[1000.],
[1100.],
[1200.],
[1300.],
[1400.],
[1500.],
[1600.],
]

a=np.asarray(a)
b=np.asarray(b)
np.linalg.cond(a)

2.586788006020457

In [1618]:
X = gauss_seidel(a,b)
X

total iteration =  15
diff =  4.588154586550976e-11
SSE =  7.582038985844803e-19


array([[-2.97652404],
       [-2.20258491],
       [ 0.21857529],
       [ 1.04337603],
       [ 3.13308531],
       [ 3.19149597],
       [ 5.76874833],
       [ 4.23922511],
       [ 6.21901116],
       [ 7.39811823],
       [ 8.63409752],
       [11.64041954],
       [13.09010808],
       [11.68341394],
       [13.19096656]])

#  Gauss-Seidel method Manual With Partial Pivoting

In [1726]:
aa=[[9 ,8 ,3 ,7 ,6 ,6 ,2 ,8 ,4 ,9 ,7 ,0 ,6 ,99 ,5],
[3 ,90 ,4 ,5 ,9 ,2 ,1 ,7 ,8 ,9 ,4 ,7 ,3 ,8 ,5],
[8 ,4 ,85 ,5 ,3 ,2 ,0 ,1 ,2 ,8 ,6 ,4 ,9 ,3 ,5],
[7 ,5 ,5 ,86 ,9 ,9 ,0 ,2 ,4 ,0 ,9 ,4 ,4 ,7 ,7],
[7 ,9 ,3 ,9 ,79 ,3 ,4 ,1 ,3 ,5 ,7 ,7 ,6 ,6 ,0],
[5 ,2 ,2 ,9 ,3 ,95 ,7 ,0 ,2 ,9 ,8 ,4 ,6 ,6 ,1],
[1 ,1 ,0 ,0 ,4 ,7 ,78 ,5 ,3 ,8 ,1 ,5 ,8 ,2 ,2],
[9 ,7 ,1 ,2 ,1 ,0 ,5 ,92 ,3 ,7 ,3 ,9 ,9 ,8 ,8],
[3 ,8 ,2 ,4 ,3 ,2 ,3 ,3 ,99 ,9 ,9 ,1 ,5 ,4 ,7],
[5 ,9 ,8 ,0 ,5 ,9 ,8 ,7 ,9 ,92 ,8 ,2 ,5 ,9 ,1],
[8 ,4 ,6 ,9 ,7 ,8 ,1 ,3 ,9 ,8 ,96 ,1 ,2 ,7 ,7],
[6 ,7 ,4 ,4 ,7 ,4 ,5 ,9 ,1 ,2 ,1 ,87 ,5 ,0 ,9],
[3 ,3 ,9 ,4 ,6 ,6 ,8 ,9 ,5 ,5 ,2 ,5 ,81 ,6 ,1],
[75 ,3 ,8 ,7 ,7 ,5 ,1 ,9 ,3 ,5 ,8 ,6 ,3 ,9 ,0],
[0 ,5 ,5 ,7 ,0 ,1 ,2 ,8 ,7 ,1 ,7 ,9 ,1 ,5 ,96]]

bb=[[1500],
[300],
[400],
[500],
[600],
[700],
[800],
[900],
[1000],
[1100],
[1200],
[1300],
[1400],
[200],
[1600]]
Bc,p=partial_pivoting(np.hstack((aa,bb)))
bb=np.asarray(bb)
cc=Bc[:,N]
for i in range (N):
    bb[i,0]=cc[i]
B,p=partial_pivoting(np.asarray(aa))
np.linalg.cond(aa)

2.5867880060204564

In [1721]:
#To ckeck if new Coeffieint Matrix can be solved or it needs pivtoing
bCheck=[[1500],
[300],
[400],
[500],
[600],
[700],
[800],
[900],
[1000],
[1100],
[1200],
[1300],
[1400],
[200],
[1600]]
xCheck=gauss_seidel(np.asarray(aa),np.asarray(bCheck))
xCheck
#It cannot be performed

C:\Users\aryas\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in square
  # This is added back by InteractiveShellApp.init_path()
C:\Users\aryas\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: overflow encountered in multiply


KeyboardInterrupt: 

In [1728]:
B,p=partial_pivoting(np.asarray(aa))
X = gauss_seidel(B,np.asarray(bb))
X

total iteration =  15
diff =  4.588154586550976e-11
SSE =  7.582038985844803e-19


array([[-2.97652404],
       [-2.20258491],
       [ 0.21857529],
       [ 1.04337603],
       [ 3.13308531],
       [ 3.19149597],
       [ 5.76874833],
       [ 4.23922511],
       [ 6.21901116],
       [ 7.39811823],
       [ 8.63409752],
       [11.64041954],
       [13.09010808],
       [11.68341394],
       [13.19096656]])

#  Successive Over-Relaxation method Manual Without Partial Pivoting

In [1729]:
class sor_ans:
    def __init__(self, x, SSE, i, diff):
            self.x = x
            self.SSE = SSE
            self.i = i
            self.diff = diff

In [1487]:
def sor_method(A,b, omega = 1, maxit = 2000):
    N = A.shape[0]
    x = np.ones((N,1))
    L = np.tril(A, k = -1)
    U = np.triu(A, k = 1)
    D = np.zeros([N,N])
    D[range(N),range(N)]  = A[range(N),range(N)]
    for i in range(maxit):
        x_old = x
        #L x = rhs
        rhs = omega * b - (omega * U + (omega-1)*D).dot(x)        
        x = forward_sub(D + omega * L,rhs,N)
        diff = (((x-x_old)**2).sum())**0.5
        if diff < 1e-10:
            print('total iteration = ', i)
            print('diff = ', diff)
            SSE = ((A.dot(x) - b)**2).sum()
            print('SSE = ', SSE)
            break
    if i < maxit-1:
        return sor_ans(x, SSE, i, diff)
    else:
        SSE = ((A.dot(x) - b)**2).sum()
        return sor_ans(x, SSE, i, diff)


In [1505]:
a=[[75 ,3 ,8 ,7 ,7 ,5 ,1 ,9 ,3 ,5 ,8 ,6 ,3 ,9 ,0],
[3 ,90 ,4 ,5 ,9 ,2 ,1 ,7 ,8 ,9 ,4 ,7 ,3 ,8 ,5],
[8 ,4 ,85 ,5 ,3 ,2 ,0 ,1 ,2 ,8 ,6 ,4 ,9 ,3 ,5],
[7 ,5 ,5 ,86 ,9 ,9 ,0 ,2 ,4 ,0 ,9 ,4 ,4 ,7 ,7],
[7 ,9 ,3 ,9 ,79 ,3 ,4 ,1 ,3 ,5 ,7 ,7 ,6 ,6 ,0],
[5 ,2 ,2 ,9 ,3 ,95 ,7 ,0 ,2 ,9 ,8 ,4 ,6 ,6 ,1],
[1 ,1 ,0 ,0 ,4 ,7 ,78 ,5 ,3 ,8 ,1 ,5 ,8 ,2 ,2],
[9 ,7 ,1 ,2 ,1 ,0 ,5 ,92 ,3 ,7 ,3 ,9 ,9 ,8 ,8],
[3 ,8 ,2 ,4 ,3 ,2 ,3 ,3 ,99 ,9 ,9 ,1 ,5 ,4 ,7],
[5 ,9 ,8 ,0 ,5 ,9 ,8 ,7 ,9 ,92 ,8 ,2 ,5 ,9 ,1],
[8 ,4 ,6 ,9 ,7 ,8 ,1 ,3 ,9 ,8 ,96 ,1 ,2 ,7 ,7],
[6 ,7 ,4 ,4 ,7 ,4 ,5 ,9 ,1 ,2 ,1 ,87 ,5 ,0 ,9],
[3 ,3 ,9 ,4 ,6 ,6 ,8 ,9 ,5 ,5 ,2 ,5 ,81 ,6 ,1],
[9 ,8 ,3 ,7 ,6 ,6 ,2 ,8 ,4 ,9 ,7 ,0 ,6 ,99 ,5],
[0 ,5 ,5 ,7 ,0 ,1 ,2 ,8 ,7 ,1 ,7 ,9 ,1 ,5 ,96]]
b=[[200.],
[300.],
[400.],
[500.],
[600.],
[700.],
[800.],
[900.],
[1000.],
[1100.],
[1200.],
[1300.],
[1400.],
[1500.],
[1600.],
]

a=np.asarray(a)
b=np.asarray(b)
np.linalg.cond(a)

2.586788006020457

In [1489]:
p=sor_method(a,b, omega = 1.01, maxit = 10000)
p.x

total iteration =  16
diff =  1.7752966768340968e-11
SSE =  3.468113126447047e-20


array([[-2.97652404],
       [-2.20258491],
       [ 0.21857529],
       [ 1.04337603],
       [ 3.13308531],
       [ 3.19149597],
       [ 5.76874833],
       [ 4.23922511],
       [ 6.21901116],
       [ 7.39811823],
       [ 8.63409752],
       [11.64041954],
       [13.09010808],
       [11.68341394],
       [13.19096656]])

#  Successive Over-Relaxation method Manual With Partial Pivoting

In [1506]:
aa=[[9 ,8 ,3 ,7 ,6 ,6 ,2 ,8 ,4 ,9 ,7 ,0 ,6 ,99 ,5],
[3 ,90 ,4 ,5 ,9 ,2 ,1 ,7 ,8 ,9 ,4 ,7 ,3 ,8 ,5],
[8 ,4 ,85 ,5 ,3 ,2 ,0 ,1 ,2 ,8 ,6 ,4 ,9 ,3 ,5],
[7 ,5 ,5 ,86 ,9 ,9 ,0 ,2 ,4 ,0 ,9 ,4 ,4 ,7 ,7],
[7 ,9 ,3 ,9 ,79 ,3 ,4 ,1 ,3 ,5 ,7 ,7 ,6 ,6 ,0],
[5 ,2 ,2 ,9 ,3 ,95 ,7 ,0 ,2 ,9 ,8 ,4 ,6 ,6 ,1],
[1 ,1 ,0 ,0 ,4 ,7 ,78 ,5 ,3 ,8 ,1 ,5 ,8 ,2 ,2],
[9 ,7 ,1 ,2 ,1 ,0 ,5 ,92 ,3 ,7 ,3 ,9 ,9 ,8 ,8],
[3 ,8 ,2 ,4 ,3 ,2 ,3 ,3 ,99 ,9 ,9 ,1 ,5 ,4 ,7],
[5 ,9 ,8 ,0 ,5 ,9 ,8 ,7 ,9 ,92 ,8 ,2 ,5 ,9 ,1],
[8 ,4 ,6 ,9 ,7 ,8 ,1 ,3 ,9 ,8 ,96 ,1 ,2 ,7 ,7],
[6 ,7 ,4 ,4 ,7 ,4 ,5 ,9 ,1 ,2 ,1 ,87 ,5 ,0 ,9],
[3 ,3 ,9 ,4 ,6 ,6 ,8 ,9 ,5 ,5 ,2 ,5 ,81 ,6 ,1],
[75 ,3 ,8 ,7 ,7 ,5 ,1 ,9 ,3 ,5 ,8 ,6 ,3 ,9 ,0],
[0 ,5 ,5 ,7 ,0 ,1 ,2 ,8 ,7 ,1 ,7 ,9 ,1 ,5 ,96]]

bb=[[1500],
[300],
[400],
[500],
[600],
[700],
[800],
[900],
[1000],
[1100],
[1200],
[1300],
[1400],
[200],
[1600]]
Bc,p=partial_pivoting(np.hstack((aa,bb)))
bb=np.asarray(bb)
cc=Bc[:,N]
for i in range (N):
    bb[i,0]=cc[i]
np.linalg.cond(aa)

2.5867880060204564

In [1732]:
#To ckeck if new Coeffieint Matrix can be solved or it needs pivtoing
bCheck=[[1500],
[300],
[400],
[500],
[600],
[700],
[800],
[900],
[1000],
[1100],
[1200],
[1300],
[1400],
[200],
[1600]]
p=sor_method(np.asarray(aa),np.asarray(bCheck), omega = 1.01, maxit = 10000)
p.x
#It cannot be doen withou partial pivoting

C:\Users\aryas\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in square
  del sys.path[0]
C:\Users\aryas\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: overflow encountered in multiply


array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan]])

In [1491]:
B,p=partial_pivoting(np.asarray(aa))
p=sor_method(B,bb, omega = 1.01, maxit = 10000)
p.x

total iteration =  16
diff =  1.7752966768340968e-11
SSE =  3.468113126447047e-20


array([[-2.97652404],
       [-2.20258491],
       [ 0.21857529],
       [ 1.04337603],
       [ 3.13308531],
       [ 3.19149597],
       [ 5.76874833],
       [ 4.23922511],
       [ 6.21901116],
       [ 7.39811823],
       [ 8.63409752],
       [11.64041954],
       [13.09010808],
       [11.68341394],
       [13.19096656]])

#  Thomas Algorithm method Manual and Scipy Without Partial Pivoting

In [1507]:
b=[[20.],
[30.],
[40.],
[50.],
[60.],
[70.],
[80.],
[90.],
[100.],
[110.],
[120.],
[130.],
[140.],
[150.],
[160.],]
N = 15
a = np.zeros((N,N))
for i in range(N):
    for j in range(N):
        if i == j:
            a[i,j] = 32
        if j == i+1:
            a[i,j] = -4
        if j == i-1:
            a[i,j] = 6
a=np.asarray(a)
b=np.asarray(b)
cc=b
np.linalg.cond(a)

1.1342620876070666

In [1493]:
def thomas(A,d):
    #imput: A = scipy.sparse.diags
    #input: d = array shape (N,)
    #output: x from Ax = d where x.shape is (N,)
    N = d.shape[0]
    #sorting array data based on the offsets
    A_sort = sorted(zip(A.data,A.offsets), key = lambda x: x[1])
    A_dat = np.array([i[0] for i in A_sort])
    a = np.empty(N)
    b = np.empty(N)
    c = np.empty(N)
    a[0] = 0
    a[1:] = A_dat[0,:-1]
    b = A_dat[1]
    c[-1] = 0
    c[:-1]= A_dat[2,1:]
    #move 0 to the first cell of A
    #move 0 to the last cell of C
    cp = np.empty(N)
    dp = np.empty(N)
    x = np.empty(N)
    cp[0] = c[0] / b[0]
    dp[0] = d[0] / b[0]
    for i in range(1,N-1):
        cp[i] = c[i] / (b[i]-a[i]*cp[i-1])
        dp[i] = (d[i] - a[i] * dp[i-1]) / (b[i]-a[i] * cp[i-1])
    i = N-1
    x[N-1] = (d[i] - a[i] * dp[i-1]) / (b[i]-a[i] * cp[i-1])
    for i in range(N-2,-1,-1):
        x[i] = dp[i] - cp[i] * x[i+1]
    return x

In [1494]:
from scipy.sparse import linalg
x = thomas(sp.sparse.dia_matrix(a),b)
a.dot(x)
#Correct

array([ 20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100., 110., 120.,
       130., 140., 150., 160.])

In [1495]:
ans = linalg.spsolve(a,b)
a.dot(ans)

C:\Users\aryas\Anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:133: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)


array([ 20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100., 110., 120.,
       130., 140., 150., 160.])

#  Thomas Algorithm method Manual and Scipy With Partial Pivoting

In [1508]:
aa=[[0., 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         6.,  32.],
       [ 6., 32., -4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  6., 32., -4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  6., 32., -4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  6., 32., -4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  6., 32., -4.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  6., 32., -4.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  6., 32., -4.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  6., 32., -4.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6., 32., -4.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6., 32., -4.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6., 32., -4.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6., 32.,
        -4.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.,
        32., -4.],
       [ 32.,  -4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0., 0.]]

bb=[[160.],
[30.],
[40.],
[50.],
[60.],
[70.],
[80.],
[90.],
[100.],
[110.],
[120.],
[130.],
[140.],
[150.],
[20.],]
Bc,p=partial_pivoting(np.hstack((aa,bb)))
bb=np.asarray(bb)
cc=Bc[:,N]
for i in range (N):
    bb[i,0]=cc[i]
B,p=partial_pivoting(np.asarray(aa))
np.linalg.cond(a)

1.1342620876070666

In [1735]:
#To ckeck if new Coeffieint Matrix can be solved or it needs pivtoing
bCheck=[[160.],
[30.],
[40.],
[50.],
[60.],
[70.],
[80.],
[90.],
[100.],
[110.],
[120.],
[130.],
[140.],
[150.],
[20.],]
thomas(sp.sparse.dia_matrix(np.asarray(aa)),np.asarray((bCheck)))

#It cannot be done without Partial Pivoting

C:\Users\aryas\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\aryas\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\aryas\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan])

In [1497]:
x = thomas(sp.sparse.dia_matrix(B),bb)
np.asarray(B).dot(x)
#Correct

array([ 20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100., 110., 120.,
       130., 140., 150., 160.])

In [1419]:
ans = linalg.spsolve(B,bb)
B.dot(ans)

C:\Users\aryas\Anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:133: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)


array([ 20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100., 110., 120.,
       130., 140., 150., 160.])